In [ ]:
!wget https://storage.yandexcloud.net/academy.ai/japan_cars_dataset.csv

--2024-06-01 17:16:35--  https://storage.yandexcloud.net/academy.ai/japan_cars_dataset.csv
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 136735 (134K) [text/csv]
Saving to: ‘japan_cars_dataset.csv’

japan_cars_dataset. 100%[===================>] 133.53K   204KB/s    in 0.7s    

2024-06-01 17:16:37 (204 KB/s) - ‘japan_cars_dataset.csv’ saved [136735/136735]



In [ ]:
import pandas as pd
cars = pd.read_csv('japan_cars_dataset.csv', sep=',')

# Удалим строки с пустыми значениями
cars = cars.dropna()

# Выводим первые 10 машин
cars.head(10)

,Unnamed: 0,price,mark,model,year,mileage,engine_capacity,transmission,drive,hand_drive,fuel
0,0,80,nissan,march,2003,80000,1240,at,2wd,rhd,gasoline
1,1,110,nissan,march,2010,53000,1200,at,2wd,rhd,gasoline
2,2,165,nissan,lafesta,2005,47690,2000,at,2wd,rhd,gasoline
3,3,190,toyota,avensis,2008,130661,1990,at,2wd,rhd,gasoline
4,4,190,daihatsu,mira,2006,66300,660,at,2wd,rhd,gasoline
5,5,190,daihatsu,mira,2004,81400,660,at,2wd,rhd,gasoline
6,8,220,nissan,march,2010,117000,1200,at,2wd,rhd,gasoline
7,9,230,volkswagen,passat,2008,127763,3190,at,4wd,rhd,gasoline
8,10,275,mazda,bongo van,2010,178218,1800,mt,2wd,rhd,gasoline
9,11,283,honda,step wgn,2005,121655,2000,at,2wd,rhd,gasoline


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

# Загрузка данных
cars = pd.read_csv('japan_cars_dataset.csv', sep=',')

In [ ]:
# Удаление строк с пустыми значениями
cars = cars.dropna()

X = cars.drop('price', axis=1)
y = cars['price']

categorical_features = X.select_dtypes(include=['object']).columns
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns

# Разделение данных на обучающую, тестовую и проверочную выборки
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:

# Обработка числовых данных
scaler = StandardScaler()
X_train_num = scaler.fit_transform(X_train[numerical_features])
X_test_num = scaler.transform(X_test[numerical_features])
X_val_num = scaler.transform(X_val[numerical_features])

# Обработка категориальных данных
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
X_train_cat = encoder.fit_transform(X_train[categorical_features])
X_test_cat = encoder.transform(X_test[categorical_features])
X_val_cat = encoder.transform(X_val[categorical_features])

# Создание модели с двумя входами
input_num = Input(shape=(X_train_num.shape[1],))
input_cat = Input(shape=(X_train_cat.shape[1],))

# Первый вход для числовых данных
x1 = Dense(64, activation="relu", kernel_regularizer=l2(0.001))(input_num)
x1 = Dense(128, activation="relu", kernel_regularizer=l2(0.001))(x1)
x1 = Dense(256, activation="relu", kernel_regularizer=l2(0.001))(x1)

# Второй вход для категориальных данных
x2 = Dense(64, activation="relu", kernel_regularizer=l2(0.001))(input_cat)
x2 = Dense(128, activation="relu", kernel_regularizer=l2(0.001))(x2)
x2 = Dropout(0.5)(x2)

# Объединение двух веток
x = concatenate([x1, x2])

# Промежуточный слой
x = Dense(64, activation='relu', kernel_regularizer=l2(0.001))(x)
x = Dropout(0.5)(x)

# Финальный регрессирующий нейрон
output = Dense(1, activation='linear')(x)
# В Model передаются входы и выход
model = Model(inputs=[input_num, input_cat], outputs=output)

# Компиляция модели
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# Обучение модели
history = model.fit([X_train_num, X_train_cat], y_train, epochs=200, batch_size=32, validation_data=([X_val_num, X_val_cat], y_val))

# Оценка модели на тестовой выборке
y_pred = model.predict([X_test_num, X_test_cat])
mse = mean_squared_error(y_test, y_pred)

print(f'Средняя квадратичная ошибка на тестовом наборе: {mse}')

# Определение среднего процента ошибки на проверочной выборке
percent_error = np.mean(np.abs((y_val - model.predict([X_val_num, X_val_cat]).flatten()) / y_val)) * 100
print(f'Средняя процентная ошибка на валидационном множестве: {percent_error:.2f}%')

Epoch 1/200


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


44/44 [==============================] - 2s 10ms/step - loss: 979895.6875 - val_loss: 695969.7500
Epoch 2/200
44/44 [==============================] - 0s 7ms/step - loss: 285550.6250 - val_loss: 71327.3438
Epoch 3/200
44/44 [==============================] - 0s 6ms/step - loss: 110622.7422 - val_loss: 34827.0234
Epoch 4/200
44/44 [==============================] - 0s 6ms/step - loss: 72103.3984 - val_loss: 21973.9941
Epoch 5/200
44/44 [==============================] - 0s 6ms/step - loss: 64024.2031 - val_loss: 17605.8242
Epoch 6/200
44/44 [==============================] - 0s 6ms/step - loss: 54700.6289 - val_loss: 14560.9561
Epoch 7/200
44/44 [==============================] - 0s 5ms/step - loss: 47884.0234 - val_loss: 11825.0850
Epoch 8/200
44/44 [==============================] - 0s 6ms/step - loss: 49620.6484 - val_loss: 12378.4688
Epoch 9/200
44/44 [==============================] - 0s 6ms/step - loss: 43986.7578 - val_loss: 13596.2480
Epoch 10/200
44/44 [========================